In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import itertools
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.cluster.util import cosine_distance
import networkx as nx

In [2]:
df = pd.read_csv('Covid-data-with-topic-names.csv')

In [3]:
df.head()

,text,event_timestamp,Text_Cleaned,Preprocessed_Text,Topic,Sub-Topic,Topic_,Sub-Topic_,month
0,Today I took advantage of these gas prices and...,2020-07-04,today i took advantage of these gas prices and...,"['today', 'took', 'price', 'never', 'given', '...",9,2,Time Periods,Daily Reports,7
1,Hey Rush Limbaugh why don t you go to your loc...,2020-11-04,hey rush limbaugh why don t you go to your loc...,"['go', 'local', 'today', 'senior', 'stage']",6,2,Store Visits,Price Awareness,11
2,She said the orders are placed over the phone ...,2020-07-04,she said the orders are placed over the phone ...,"['said', 'order', 'place', 'phone', 'via', 'lo...",4,1,Online Shopping,Order Pricing,7
3,COVID-19 has wiped the calendars of experienti...,2020-10-04,covid 19 has wiped the calendars of experienti...,"['live', 'event', 'agenc', 'futur', 'forc', 'b...",5,1,Consumer Impact,Market Impact,10
4,The consumer s mindset was already showing sig...,2020-10-04,the consumer s mindset was already showing sig...,"['consum', 'alreadi', 'show', 'sign', 'could',...",5,1,Consumer Impact,Market Impact,10


In [4]:
df['Topic'].value_counts()

Topic
5    1513
6    1455
0    1078
3     969
1     954
7     922
9     536
4     499
2     493
8     429
Name: count, dtype: int64

In [7]:
df['Preprocessed_Text'] = df['Preprocessed_Text'].apply(lambda x: eval(x))

In [8]:
import pandas as pd
import json
import networkx as nx
from collections import defaultdict
# Calculate TextRank for each sub-topic
textranks = defaultdict(lambda: defaultdict(dict))

for index, row in df.iterrows():
    topic = row['Topic_']
    sub_topic = row['Sub-Topic_']
    preprocessed_text = row['Preprocessed_Text']


    # Build a graph for TextRank
    G = nx.Graph()
    G.add_nodes_from(preprocessed_text)

    # Calculate TextRank scores
    for i, token_i in enumerate(preprocessed_text):
        for j, token_j in enumerate(preprocessed_text):
            if i != j:
                G.add_edge(token_i, token_j)

    # Run TextRank
    scores = nx.pagerank(G)

    # Store TextRank scores for tokens in the hierarchical structure
    for token, score in scores.items():
        textranks[topic][sub_topic][token] = score

# Convert the hierarchical structure to JSON
json_output = json.dumps(textranks, indent=4)
print(json_output)

{
    "Time Periods": {
        "Daily Reports": {
            "today": 0.07142857142857142,
            "took": 0.09906439106652945,
            "price": 0.2,
            "never": 0.06249999999999999,
            "given": 0.06249999999999999,
            "mani": 0.14002333769825887,
            "look": 0.06249999999999999,
            "life": 0.3333333333333333,
            "help": 0.12499999999999997,
            "new": 0.1098146855814067,
            "case": 0.10842048040123452,
            "last": 0.14002333769825887,
            "day": 0.3333333333333333,
            "keep": 0.07142857142857142,
            "good": 0.11111111111111113,
            "work": 0.12499999999999997,
            "get": 0.1098146855814067,
            "us": 0.07692307692307693,
            "test": 0.062279427019472484,
            "start": 0.09906439106652945,
            "report": 0.5,
            "number": 0.09906439106652945,
            "commun": 0.055401663237311397,
            "believ": 0.0622794270

In [9]:
with open('textrank_output.json', 'w') as f:
    f.write(json_output)

